In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Phase 1: Data Exploration & Analysis\n",
    "\n",
    "**Real Estate Value Analyzer - Miami Property Data**\n",
    "\n",
    "This notebook explores our three data sources:\n",
    "1. Miami Housing Data (detailed property records)\n",
    "2. Median Listing Prices (historical trends)\n",
    "3. Zillow Home Value Index (market indicators)\n",
    "\n",
    "**Goals:**\n",
    "- Understand data quality and structure\n",
    "- Identify key features for price prediction\n",
    "- Prepare data for ML model\n",
    "- Create market insights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.metrics import mean_absolute_error, r2_score\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set style\n",
    "plt.style.use('default')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "print(\"✅ Libraries imported successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load and Explore Miami Housing Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load Miami housing data\n",
    "miami_df = pd.read_csv('../data/miami-housing.csv')\n",
    "\n",
    "print(f\"📊 Miami Housing Dataset Shape: {miami_df.shape}\")\n",
    "print(f\"\\n📋 Columns: {list(miami_df.columns)}\")\n",
    "print(f\"\\n💰 Price Range: ${miami_df['SALE_PRC'].min():,.0f} - ${miami_df['SALE_PRC'].max():,.0f}\")\n",
    "print(f\"🏠 Average Price: ${miami_df['SALE_PRC'].mean():,.0f}\")\n",
    "print(f\"📏 Average Square Footage: {miami_df['TOT_LVG_AREA'].mean():.0f} sq ft\")\n",
    "\n",
    "# Display first few rows\n",
    "miami_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data quality check\n",
    "print(\"🔍 Data Quality Analysis:\")\n",
    "print(f\"Missing values:\\n{miami_df.isnull().sum()}\")\n",
    "print(f\"\\nData types:\\n{miami_df.dtypes}\")\n",
    "print(f\"\\nNumeric columns summary:\")\n",
    "miami_df.describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load Historical Price Trends"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load median listing prices\n",
    "price_trends = pd.read_csv('../data/MEDLISPRI12086.csv')\n",
    "price_trends['observation_date'] = pd.to_datetime(price_trends['observation_date'])\n",
    "\n",
    "print(f\"📈 Price Trends Dataset Shape: {price_trends.shape}\")\n",
    "print(f\"📅 Date Range: {price_trends['observation_date'].min()} to {price_trends['observation_date'].max()}\")\n",
    "print(f\"💰 Price Range: ${price_trends['MEDLISPRI12086'].min():,.0f} - ${price_trends['MEDLISPRI12086'].max():,.0f}\")\n",
    "\n",
    "price_trends.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Feature Engineering & Data Preparation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create features for ML model\n",
    "def prepare_features(df):\n",
    "    \"\"\"Prepare features for price prediction model\"\"\"\n",
    "    \n",
    "    # Create a copy to avoid modifying original\n",
    "    df_clean = df.copy()\n",
    "    \n",
    "    # Handle missing values\n",
    "    df_clean = df_clean.dropna(subset=['SALE_PRC', 'TOT_LVG_AREA'])\n",
    "    \n",
    "    # Create new features\n",
    "    df_clean['price_per_sqft'] = df_clean['SALE_PRC'] / df_clean['TOT_LVG_AREA']\n",
    "    df_clean['land_to_building_ratio'] = df_clean['LND_SQFOOT'] / df_clean['TOT_LVG_AREA']\n",
    "    \n",
    "    # Distance features (log transform for better distribution)\n",
    "    df_clean['log_ocean_dist'] = np.log1p(df_clean['OCEAN_DIST'])\n",
    "    df_clean['log_water_dist'] = np.log1p(df_clean['WATER_DIST'])\n",
    "    df_clean['log_center_dist'] = np.log1p(df_clean['CNTR_DIST'])\n",
    "    \n",
    "    # Quality features\n",
    "    df_clean['has_special_features'] = (df_clean['SPEC_FEAT_VAL'] > 0).astype(int)\n",
    "    \n",
    "    # Age categories\n",
    "    df_clean['age_category'] = pd.cut(df_clean['age'], \n",
    "                                     bins=[0, 10, 25, 50, 100], \n",
    "                                     labels=['New', 'Recent', 'Mature', 'Historic'])\n",
    "    \n",
    "    return df_clean\n",
    "\n",
    "# Prepare features\n",
    "miami_processed = prepare_features(miami_df)\n",
    "\n",
    "print(f\"✅ Processed dataset shape: {miami_processed.shape}\")\n",
    "print(f\"💰 Average price per sq ft: ${miami_processed['price_per_sqft'].mean():.2f}\")\n",
    "print(f\"🏠 Average building size: {miami_processed['TOT_LVG_AREA'].mean():.0f} sq ft\")\n",
    "\n",
    "# Show new features\n",
    "new_features = ['price_per_sqft', 'land_to_building_ratio', 'log_ocean_dist', \n",
    "                'log_water_dist', 'log_center_dist', 'has_special_features', 'age_category']\n",
    "miami_processed[new_features + ['SALE_PRC']].head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Market Analysis & Visualizations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comprehensive market analysis\n",
    "fig = make_subplots(\n",
    "    rows=2, cols=2,\n",
    "    subplot_titles=('Price Distribution', 'Price vs Square Footage', \n",
    "                   'Price by Structure Quality', 'Historical Price Trends'),\n",
    "    specs=[[{\"secondary_y\": False}, {\"secondary_y\": False}],\n",
    "           [{\"secondary_y\": False}, {\"secondary_y\": False}]]\n",
    ")\n",
    "\n",
    "# 1. Price distribution\n",
    "fig.add_trace(\n",
    "    go.Histogram(x=miami_processed['SALE_PRC'], nbinsx=50, name='Price Distribution'),\n",
    "    row=1, col=1\n",
    ")\n",
    "\n",
    "# 2. Price vs Square Footage\n",
    "fig.add_trace(\n",
    "    go.Scatter(x=miami_processed['TOT_LVG_AREA'], y=miami_processed['SALE_PRC'], \n",
    "               mode='markers', marker=dict(size=3, opacity=0.6), name='Properties'),\n",
    "    row=1, col=2\n",
    ")\n",
    "\n",
    "# 3. Price by Structure Quality\n",
    "quality_stats = miami_processed.groupby('structure_quality')['SALE_PRC'].agg(['mean', 'count']).reset_index()\n",
    "fig.add_trace(\n",
    "    go.Bar(x=quality_stats['structure_quality'], y=quality_stats['mean'], \n",
    "           name='Avg Price by Quality'),\n",
    "    row=2, col=1\n",
    ")\n",
    "\n",
    "# 4. Historical trends\n",
    "fig.add_trace(\n",
    "    go.Scatter(x=price_trends['observation_date'], y=price_trends['MEDLISPRI12086'], \n",
    "               mode='lines+markers', name='Median Price'),\n",
    "    row=2, col=2\n",
    ")\n",
    "\n",
    "fig.update_layout(height=800, title_text=\"Miami Real Estate Market Analysis\", showlegend=False)\n",
    "fig.show()\n",
    "\n",
    "# Save the plot\n",
    "fig.write_html('../docs/market_analysis.html')\n",
    "print(\"📊 Market analysis saved to docs/market_analysis.html\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Price Prediction Model Development"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Select features for ML model\n",
    "feature_columns = [\n",
    "    'TOT_LVG_AREA', 'LND_SQFOOT', 'SPEC_FEAT_VAL', 'age',\n",
    "    'OCEAN_DIST', 'WATER_DIST', 'CNTR_DIST', 'structure_quality',\n",
    "    'price_per_sqft', 'land_to_building_ratio', 'log_ocean_dist',\n",
    "    'log_water_dist', 'log_center_dist', 'has_special_features'\n",
    "]\n",
    "\n",
    "# Prepare data for modeling\n",
    "X = miami_processed[feature_columns].copy()\n",
    "y = miami_processed['SALE_PRC']\n",
    "\n",
    "# Handle any remaining missing values\n",
    "X = X.fillna(X.median())\n",
    "\n",
    "# Split data\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n",
    "\n",
    "print(f\"📊 Training set: {X_train.shape[0]} properties\")\n",
    "print(f\"📊 Test set: {X_test.shape[0]} properties\")\n",
    "print(f\"💰 Target variable range: ${y.min():,.0f} - ${y.max():,.0f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train Random Forest model\n",
    "rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)\n",
    "rf_model.fit(X_train, y_train)\n",
    "\n",
    "# Make predictions\n",
    "y_pred = rf_model.predict(X_test)\n",
    "\n",
    "# Evaluate model\n",
    "mae = mean_absolute_error(y_test, y_pred)\n",
    "r2 = r2_score(y_test, y_pred)\n",
    "\n",
    "print(\"🎯 Model Performance:\")\n",
    "print(f\"Mean Absolute Error: ${mae:,.0f}\")\n",
    "print(f\"R² Score: {r2:.3f}\")\n",
    "print(f\"Average Price: ${y_test.mean():,.0f}\")\n",
    "print(f\"Error as % of avg price: {(mae/y_test.mean())*100:.1f}%\")\n",
    "\n",
    "# Feature importance\n",
    "feature_importance = pd.DataFrame({\n",
    "    'feature': feature_columns,\n",
    "    'importance': rf_model.feature_importances_\n",
    "}).sort_values('importance', ascending=False)\n",
    "\n",
    "print(\"\\n🔍 Top 10 Most Important Features:\")\n",
    "print(feature_importance.head(10))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save model and feature importance\n",
    "import joblib\n",
    "import json\n",
    "\n",
    "# Save model\n",
    "joblib.dump(rf_model, '../docs/price_prediction_model.pkl')\n",
    "\n",
    "# Save feature importance\n",
    "feature_importance.to_csv('../docs/feature_importance.csv', index=False)\n",
    "\n",
    "# Save model metadata\n",
    "model_metadata = {\n",
    "    'model_type': 'RandomForestRegressor',\n",
    "    'features': feature_columns,\n",
    "    'mae': float(mae),\n",
    "    'r2_score': float(r2),\n",
    "    'training_samples': len(X_train),\n",
    "    'test_samples': len(X_test),\n",
    "    'avg_price': float(y_test.mean()),\n",
    "    'error_percentage': float((mae/y_test.mean())*100)\n",
    "}\n",
    "\n",
    "with open('../docs/model_metadata.json', 'w') as f:\n",
    "    json.dump(model_metadata, f, indent=2)\n",
    "\n",
    "print(\"✅ Model and metadata saved to docs/ folder\")\n",
    "print(f\"📊 Model R² Score: {r2:.3f}\")\n",
    "print(f\"💰 Average Error: ${mae:,.0f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Market Insights Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate market insights\n",
    "insights = {\n",
    "    'total_properties': len(miami_processed),\n",
    "    'avg_price': miami_processed['SALE_PRC'].mean(),\n",
    "    'median_price': miami_processed['SALE_PRC'].median(),\n",
    "    'avg_sqft': miami_processed['TOT_LVG_AREA'].mean(),\n",
    "    'avg_price_per_sqft': miami_processed['price_per_sqft'].mean(),\n",
    "    'price_range': {\n",
    "        'min': miami_processed['SALE_PRC'].min(),\n",
    "        'max': miami_processed['SALE_PRC'].max()\n",
    "    },\n",
    "    'model_performance': {\n",
    "        'mae': mae,\n",
    "        'r2_score': r2,\n",
    "        'error_percentage': (mae/y_test.mean())*100\n",
    "    },\n",
    "    'top_features': feature_importance.head(5)['feature'].tolist()\n",
    "}\n",
    "\n",
    "print(\"📊 Miami Real Estate Market Insights:\")\n",
    "print(f\"🏠 Total Properties Analyzed: {insights['total_properties']:,}\")\n",
    "print(f\"💰 Average Price: ${insights['avg_price']:,.0f}\")\n",
    "print(f\"📏 Average Size: {insights['avg_sqft']:.0f} sq ft\")\n",
    "print(f\"💵 Average Price per Sq Ft: ${insights['avg_price_per_sqft']:.2f}\")\n",
    "print(f\"🎯 Model Accuracy (R²): {insights['model_performance']['r2_score']:.3f}\")\n",
    "print(f\"🔍 Top Features: {', '.join(insights['top_features'])}\")\n",
    "\n",
    "# Save insights\n",
    "with open('../docs/market_insights.json', 'w') as f:\n",
    "    json.dump(insights, f, indent=2)\n",
    "\n",
    "print(\"\\n✅ Phase 1 Complete! Ready for UI development.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}